---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind
import re

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [2]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [143]:
def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    university_towns = pd.read_table('university_towns.txt', header = None)
    university_towns.columns = ['RegionName']
    university_towns['State'] = np.nan
    for town in university_towns['RegionName']:
        if re.search('(\[edit\])', town): 
            university_towns.State[university_towns.RegionName == town] = town
    university_towns.State = university_towns.State.fillna(method = 'ffill')
    university_towns = university_towns[university_towns.RegionName != university_towns.State]
    university_towns = university_towns.reset_index(drop = True)
    university_towns.RegionName = university_towns.RegionName.str.replace(r'[ \(\[:].*','')
    university_towns.State = university_towns.State.str.replace(r'[\[:].*\]','')
    university_towns = university_towns[['State','RegionName']]
    return university_towns
get_list_of_university_towns()


,State,RegionName
0,Alabama,Auburn
1,Alabama,Florence
2,Alabama,Jacksonville
3,Alabama,Livingston
4,Alabama,Montevallo
5,Alabama,Troy
6,Alabama,Tuscaloosa
7,Alabama,Tuskegee
8,Alaska,Fairbanks
9,Arizona,Flagstaff


In [4]:
# university_towns = pd.read_table('university_towns.txt', header = None)
# university_towns.columns = ['RegionName']
# university_towns['State'] = np.nan
# for town in university_towns['RegionName']:
#     if re.search('(\[edit\])', town): 
#         university_towns.State[university_towns.RegionName == town] = town
# university_towns.State = university_towns.State.fillna(method = 'ffill')
# university_towns = university_towns[university_towns.RegionName != university_towns.State]
# university_towns = university_towns.reset_index(drop = True)
# university_towns.RegionName = university_towns.RegionName.str.replace(r'[\(\[:].*','')
# university_towns.State = university_towns.State.str.replace(r'[\[:].*\]','')
# university_towns.set_index('State',inplace=True)
# university_towns.head()

In [5]:
def GDP_file():
    GDP = pd.read_excel('gdplev.xls', skiprows = 5)
    GDP = GDP.drop(['Unnamed: 0','GDP in billions of current dollars',\
                'GDP in billions of chained 2009 dollars','Unnamed: 3','Unnamed: 7'], axis = 1).drop([0,1])
    GDP.columns = ['Quater','GDP in billions of current dollars','GDP in billions of chained 2009 dollars']
    GDP = GDP[GDP.Quater.str.startswith('2')]
    GDP = GDP.reset_index(drop = True)
    GDP['Diff'] = GDP['GDP in billions of current dollars'].diff()
    return GDP
GDP_file()

,Quater,GDP in billions of current dollars,GDP in billions of chained 2009 dollars,Diff
0,2000q1,10031.0,12359.1,NaN
1,2000q2,10278.3,12592.5,247.3
2,2000q3,10357.4,12607.7,79.1
3,2000q4,10472.3,12679.3,114.9
4,2001q1,10508.1,12643.3,35.8
5,2001q2,10638.4,12710.3,130.3
6,2001q3,10639.5,12670.1,1.1
7,2001q4,10701.3,12705.3,61.8
8,2002q1,10834.4,12822.3,133.1
9,2002q2,10934.8,12893.0,100.4


In [8]:
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    GDP = GDP_file()
    return GDP[(GDP['Diff']>0) & (GDP['Diff'].shift(-1)<0) & (GDP['Diff'].shift(-2)<0)].Quater.iloc[0]
get_recession_start()

'2008q3'

In [27]:
# GDP = pd.read_excel('gdplev.xls', skiprows = 5)
# GDP = GDP.drop(['Unnamed: 0','GDP in billions of current dollars',\
#                 'GDP in billions of chained 2009 dollars','Unnamed: 3','Unnamed: 7'], axis = 1).drop([0,1])
# GDP.columns = ['Quater','GDP in billions of current dollars','GDP in billions of chained 2009 dollars']
# GDP = GDP[GDP.Quater.str.startswith('2')]
# GDP = GDP.reset_index(drop = True)
# GDP['Diff'] = GDP['GDP in billions of current dollars'].diff()
# GDP.head(60)

,Quater,GDP in billions of current dollars,GDP in billions of chained 2009 dollars,Diff
0,2000q1,10031.0,12359.1,NaN
1,2000q2,10278.3,12592.5,247.3
2,2000q3,10357.4,12607.7,79.1
3,2000q4,10472.3,12679.3,114.9
4,2001q1,10508.1,12643.3,35.8
5,2001q2,10638.4,12710.3,130.3
6,2001q3,10639.5,12670.1,1.1
7,2001q4,10701.3,12705.3,61.8
8,2002q1,10834.4,12822.3,133.1
9,2002q2,10934.8,12893.0,100.4


In [34]:
# GDP[(GDP['Diff']>0) & (GDP['Diff'].shift(-1)<0) & (GDP['Diff'].shift(-2)<0)].Quater.iloc[0]

'2008q3'

In [9]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
    GDP = GDP_file()
    return GDP[(GDP['Diff'].shift(2)<0) & (GDP['Diff'].shift(1)>0) & (GDP['Diff']>0)].where(GDP.Quater>'2008q4').Quater.iloc[1]
get_recession_end()

'2009q4'

In [10]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    GDP = GDP_file()
    GDP_bottom = GDP['GDP in billions of current dollars'][(GDP['Quater']>=get_recession_start()) \
                                                           & (GDP['Quater']<=get_recession_end())].min()
    return GDP['Quater'][GDP['GDP in billions of current dollars']==GDP_bottom].iloc[0]
get_recession_bottom()

'2009q2'

In [55]:
# GDP = pd.read_excel('gdplev.xls', skiprows = 5)
# GDP = GDP.drop(['Unnamed: 0','GDP in billions of current dollars',\
#                 'GDP in billions of chained 2009 dollars','Unnamed: 3','Unnamed: 7'], axis = 1).drop([0,1])
# GDP.columns = ['Quater','GDP in billions of current dollars','GDP in billions of chained 2009 dollars']
# GDP = GDP[GDP.Quater.str.startswith('2')]
# GDP = GDP.reset_index(drop = True)
# GDP['Diff'] = GDP['GDP in billions of current dollars'].diff()
# GDP_bottom = GDP['GDP in billions of current dollars'][(GDP['Quater']>=get_recession_start()) & (GDP['Quater']<=get_recession_end())].min()
# GDP['Quater'][GDP['GDP in billions of current dollars']==GDP_bottom].iloc[0]
# GDP.head(60)

In [ ]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    
    return "ANSWER"

In [79]:
Zillow = pd.read_csv('City_Zhvi_AllHomes.csv')
Zillow = Zillow.drop(['Metro', 'CountyName', 'SizeRank'], axis = 1)
Column_to_drop = Zillow.columns[Zillow.columns.str.startswith('19')]
Column_to_drop
Zillow = Zillow.drop(Column_to_drop, axis = 1)
Zillow.State = Zillow.State.apply(lambda x: states[x])
# Reshape long
Zillow = Zillow.set_index(['State', 'RegionName', 'RegionID'], drop = True) 
Zillow.columns.name = 'Month'
Zillow.head()


,,Month,2000-01,2000-02,2000-03,2000-04,2000-05,2000-06,2000-07,2000-08,2000-09,2000-10,...,2015-11,2015-12,2016-01,2016-02,2016-03,2016-04,2016-05,2016-06,2016-07,2016-08
State,RegionName,RegionID,,,,,,,,,,,,,,,,,,,,,
New York,New York,6181,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,573600,576200,578400,582200,588000,592200,592500,590200,588000,586400
California,Los Angeles,12447,204400.0,207000.0,209800.0,212300.0,214500.0,216600.0,219000.0,221100.0,222800.0,224300.0,...,558200,560800,562800,565600,569700,574000,577800,580600,583000,585100
Illinois,Chicago,17426,136800.0,138300.0,140100.0,141900.0,143700.0,145300.0,146700.0,147900.0,149000.0,150400.0,...,207800,206900,206200,205800,206200,207300,208200,209100,211000,213000
Pennsylvania,Philadelphia,13271,52700.0,53100.0,53200.0,53400.0,53700.0,53800.0,53800.0,54100.0,54500.0,54700.0,...,122300,121600,121800,123300,125200,126400,127000,127400,128300,129100
Arizona,Phoenix,40326,111000.0,111700.0,112800.0,113700.0,114300.0,115100.0,115600.0,115900.0,116500.0,117200.0,...,183800,185300,186600,188000,189100,190200,191300,192800,194500,195900


In [77]:
Zillow[['Alabama','Montevallo']]

KeyError: "['Alabama' 'Montevallo'] not in index"

In [146]:
# Remove duplicate samples
# x = Zillow.drop_duplicates(subset = 'RegionID', keep = 'first')
x = Zillow.stack()
x.name = 'Value'
x = x.reset_index()
x.Month = pd.to_datetime(x.Month)
x['Year'] = x.Month.dt.year
x['Quarter'] = x.Month.dt.quarter

# Average monthly prices within each quarter
x = x.groupby(['State', 'RegionName', 'RegionID', 'Year', 'Quarter'],).mean()['Value']
x = x.reset_index()
x['Date'] = x.Year.apply(str) + 'q' + x.Quarter.apply(str)
x = x.drop(['Year', 'Quarter'], axis = 1)
x = x.set_index(['State', 'RegionName', 'RegionID', 'Date'])
x = x['Value']
x = x.unstack('Date')
x = x.reset_index()
x = x.drop('RegionID', axis = 1)
x = x.set_index(['State', 'RegionName'])
x.shape

(10730, 67)

In [ ]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    
    return "ANSWER"

In [150]:
u_towns = get_list_of_university_towns()
u_towns = u_towns[['State', 'RegionName']]
u_towns['U_Town'] = 1
u_towns = u_towns.set_index(['State', 'RegionName'])
# Merge university towns with housing prices by state and region name
x1= pd.merge(x, u_towns, how = 'left', left_index = True, right_index = True)
x1.U_Town[x1.U_Town.isnull()] = 0
x1.head(90)
    
# Analyze trends in housing values
start = get_recession_start()
bottom = get_recession_bottom()
y = x1[[start, bottom, 'U_Town']]
y['Change'] = y[bottom] / y[start]

ut = y[y['U_Town'] == 1]['Change'].dropna()
print(ut.shape)
not_ut = y[y['U_Town'] == 0]['Change'].dropna()
print(not_ut)
t = ttest_ind(ut, not_ut)
p = t[1]
    
different = False
if p < 0.01:
    different = True
    
better = 'university town'
if ut.mean() < not_ut.mean():
    better = 'non-university town'
print(different, p, better)

(211,)
State      RegionName     
Alabama    Adamsville         1.010605
           Alabaster          0.963519
           Albertville        0.988281
           Arab               1.059058
           Ardmore            0.965124
           Axis               0.930249
           Bay Minette        0.963190
           Bayou La Batre     1.047009
           Birmingham         0.943856
           Boaz               0.990119
           Brent              0.889074
           Brookwood          0.977157
           Buhl               0.967385
           Calera             0.969403
           Center Point       0.936247
           Centreville        0.876261
           Chalkville         0.958924
           Chelsea            0.937667
           Chickasaw          0.915785
           Chunchula          0.954392
           Citronelle         0.974208
           Clay               0.943484
           Coden              0.938051
           Coker              0.965143
           Concord            